In [3]:
from torch import device
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import sys, os, datetime
import json
import torch
import random
import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TranslationPipeline
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
# load the pretrained model
import languages
device = device('cpu')


source_langs = set(["akk", "sux"])

target_langs = set(["en"])

model_path = "../results/t5-small-bi-akksux-en-20240111-203816-10-8-last"
new_model_path = "../results/t5-small-bi-akksux-en-20240112-074206-30-8-last"
tokenizer_path = "../results/t5-small-pre-p-l-akksux-en-20240108-221858"


In [4]:
# get dataset
def get_prefix(src_lang, tgt_lang):
    s = languages.all_languages[src_lang]
    t = languages.all_languages[tgt_lang]
    return f"translate {s} to {t}: "


get_prefix("suxts", "es")
def load_translations_dataset(from_lang, to_lang):
    ds = Dataset.from_json(f"../data/translations_{from_lang}_to_{to_lang}.jsonl")
    srcs = [get_prefix(from_lang, to_lang) + x[from_lang] for x in ds]
    targets = [x[to_lang] for x in ds]
    srcs.extend([get_prefix(to_lang, from_lang) + x[to_lang] for x in ds])
    targets.extend([x[from_lang] for x in ds])
    #     ds = ds.add_column("source", srcs)
    #     ds = ds.add_column("target", targets)
    #     ds = ds.remove_columns([from_lang])
    #     ds = ds.rename_column(to_lang, "target")
    ds = Dataset.from_dict({"source": srcs, "target": targets})
    return ds


translation_datasets = {lang: load_translations_dataset(lang, "en") for lang in source_langs}


{'akk': Dataset({
     features: ['source', 'target'],
     num_rows: 191258
 }),
 'sux': Dataset({
     features: ['source', 'target'],
     num_rows: 42350
 })}

In [5]:
translation_datasets["sux"][1]


{'source': 'translate Sumerian to English: # (gesz)isimu3(+mu2) - |_gisz_-(U)-_ad-sar_| = %a pe-er-hu-um # (gesz)ildagx(|A-(_gu4_xKUR)|) - |_gisz-a-gu4_xKUR| = %a a-da-ru-um # (gesz)szinig - |_gisz-szinig_|# = %a bi-nu-um # (gesz)asal2 - |_gisz-a-tu-gaba-lisz_|# = %a s,a-ar-ba-tum!(_lum_) # - [...] = %a [...] sza# _dingir#_',
 'target': 'bud poplar tamarisk poplar divine weapon'}

In [9]:

merged_dataset = concatenate_datasets(translation_datasets.values()).shuffle()
akk_dataset = translation_datasets["akk"].shuffle()
sux_dataset = translation_datasets["sux"].shuffle()

merged_dataset, akk_dataset, sux_dataset

(Dataset({
     features: ['source', 'target'],
     num_rows: 233608
 }),
 Dataset({
     features: ['source', 'target'],
     num_rows: 191258
 }),
 Dataset({
     features: ['source', 'target'],
     num_rows: 42350
 }))

IndexError: index out of range in self